In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
pip install \
   pandas matplotlib numpy \
   nltk seaborn sklearn gensim pyldavis \
   wordcloud textblob spacy textstat

In [ ]:
import pandas as pd
df = pd.read_csv('/content/gdrive/My Drive/Mestrado/sentiment_dataset.csv', encoding='latin-1')
df['sentiment'].fillna('', inplace=True)

In [ ]:
df['text'][0].encode().decode()

In [ ]:
!pip install unidecode

In [ ]:
import unidecode
word = unidecode.unidecode("Â\xa0")
print(word)

In [ ]:
import re
def repair(s):
  slices = []
  for match in re.finditer('\w\.\w', s):
      slices.append(match.start()+2)
  slices.append(len(s))
  offset = 0
  subsentences = []
  for pos in sorted(slices):
      subsent = s[offset:pos]
      offset += len(subsent)
      subsentences.append(subsent)
  return ' '.join(subsentences)

In [ ]:
df['text'] = df['text'].apply(repair)

In [ ]:
def dp(text):
 return text.replace(u'\xa0', u' ')

df['text'] = df['text'].apply(unidecode.unidecode)

In [ ]:
df['text'][0]

In [ ]:
 from ast import literal_eval
 df['labels'] = df['labels'].apply(literal_eval)

In [ ]:
!pip install upsetplot

In [ ]:
df['text'].apply(lambda x : len(x.split())).hist()
plt.xlabel('Number of words')
plt.ylabel('Frequency')

In [ ]:
from nltk.tokenize import sent_tokenize,wordpunct_tokenize
import numpy as np
import nltk
nltk.download('punkt')

c=[]
def okok(text):
  c = []
  for x in sent_tokenize(text):
    c.append(len((wordpunct_tokenize(x))))
  return np.mean(c)

df['text'].apply(okok).hist()
plt.xlabel('Average sentence length/ Document')
plt.ylabel('Frequency')

In [ ]:
import matplotlib.pyplot as plt

plt.boxplot(df['text'].apply(okok))
plt.ylabel('Number of sentences')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off


In [ ]:
import numpy as np
arr1  =list(df['text'].apply(okok))
q1 = np.quantile(arr1, 0.25)
 
# finding the 3rd quartile
q3 = np.quantile(arr1, 0.75)
med = np.median(arr1)
 
# finding the iqr region
iqr = q3-q1
 
# finding upper and lower whiskers
upper_bound = q3+(1.5*iqr)
lower_bound = q1-(1.5*iqr)
print(iqr, upper_bound, lower_bound)

In [ ]:
(df['text'].apply(okok) > 21.125).sum()

In [ ]:
import collections
from collections import defaultdict

c = defaultdict(int)

for labels in df['labels']:
  labels.sort()
  c[tuple(set(labels))] += 1

In [ ]:
c = {k: v for k, v in c.items() if v > 1}

In [ ]:
keys = [list(x) for x in c.keys()]

In [ ]:
keys

In [ ]:
data = list(c.values())

In [ ]:
print(len(data))

In [ ]:
data

In [ ]:
sum(data)

In [ ]:
from upsetplot import plot
from upsetplot import from_memberships

example = from_memberships(keys, data=data)
from upsetplot import plot
plot(example)  
from matplotlib import pyplot
pyplot.show()  

In [ ]:
example.duplicated()

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df.labels)

In [ ]:
from collections import defaultdict

df2 = df

df2['len1'] = df['text'].apply(len)
df2['len2'] = df['labels'].apply(len)
  

In [ ]:
from scipy.stats.stats import pearsonr

pearsonr(df2['len1'], df2['len2'])

In [ ]:
df.iloc[[0,1]]

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df.labels)

In [ ]:
import numpy as np
np.random.seed(3) # for reproducibility

import matplotlib.pyplot as plt


def show_values(pc, fmt="%.2f", **kw):
    '''
    Heatmap with text in each cell with matplotlib's pyplot
    Source: http://stackoverflow.com/a/25074150/395857 
    By HYRY
    '''
    #from itertools import zip
    pc.update_scalarmappable()
    ax = pc.axes
    for p, color, value in zip(pc.get_paths(), pc.get_facecolors(), pc.get_array()):
        x, y = p.vertices[:-2, :].mean(0)
        if np.all(color[:3] > 0.5):
            color = (0.0, 0.0, 0.0)
        else:
            color = (1.0, 1.0, 1.0)
        ax.text(x, y, fmt % value, ha="center", va="center", color=color, **kw)

def cm2inch(*tupl):
    '''
    Specify figure size in centimeter in matplotlib
    Source: http://stackoverflow.com/a/22787457/395857
    By gns-ank
    '''
    inch = 2.54
    if type(tupl[0]) == tuple:
        return tuple(i/inch for i in tupl[0])
    else:
        return tuple(i/inch for i in tupl)



def heatmap(AUC, title, xlabel, ylabel, xticklabels, yticklabels):
    '''
    Inspired by:
    - http://stackoverflow.com/a/16124677/395857 
    - http://stackoverflow.com/a/25074150/395857
    '''

    # Plot it out
    fig, ax = plt.subplots()    
    c = ax.pcolor(AUC, edgecolors='k', linestyle= 'dashed', linewidths=0.2, cmap='RdBu', vmin=0.0, vmax=1.0)

    # put the major ticks at the middle of each cell
    ax.set_yticks(np.arange(AUC.shape[0]) + 0.5, minor=False)
    ax.set_xticks(np.arange(AUC.shape[1]) + 0.5, minor=False)

    # set tick labels
    #ax.set_xticklabels(np.arange(1,AUC.shape[1]+1), minor=False)
    ax.set_xticklabels(xticklabels, minor=False)
    ax.set_yticklabels(yticklabels, minor=False)

    # set title and x/y labels
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)      

    # Remove last blank column
    plt.xlim( (0, AUC.shape[1]) )

    # Turn off all the ticks
    ax = plt.gca()    
    for t in ax.xaxis.get_major_ticks():
        t.tick1On = False
        t.tick2On = False
    for t in ax.yaxis.get_major_ticks():
        t.tick1On = False
        t.tick2On = False

    # Add color bar
    plt.colorbar(c)

    # Add text in each cell 
    show_values(c)

    # Proper orientation (origin at the top left instead of bottom left)
    ax.invert_yaxis()
    ax.xaxis.tick_top()

    # resize 
    fig = plt.gcf()
    fig.set_size_inches(cm2inch(40, 20))

label_data = y
label_headers = mlb.classes_
cooccurrence_matrix =  y.T.dot(y) 
print('\ncooccurrence_matrix:\n{0}'.format(cooccurrence_matrix)) 

# Compute cooccurrence matrix in percentage
# FYI: http://stackoverflow.com/questions/19602187/numpy-divide-each-row-by-a-vector-element
#      http://stackoverflow.com/questions/26248654/numpy-return-0-with-divide-by-zero/32106804#32106804
cooccurrence_matrix_diagonal = np.diagonal(cooccurrence_matrix)
with np.errstate(divide='ignore', invalid='ignore'):
    cooccurrence_matrix_percentage = cooccurrence_matrix #np.nan_to_num(np.true_divide(cooccurrence_matrix, cooccurrence_matrix_diagonal[:, None]))
print('\ncooccurrence_matrix_percentage:\n{0}'.format(cooccurrence_matrix_percentage))

# Add count in labels
label_header_with_count = [ '{0} ({1})'.format(label_header, cooccurrence_matrix_diagonal[label_number]) for label_number, label_header in enumerate(label_headers)]  
print('\nlabel_header_with_count: {0}'.format(label_header_with_count))

# Plotting
x_axis_size = cooccurrence_matrix_percentage.shape[0]
y_axis_size = cooccurrence_matrix_percentage.shape[1]
title = "Co-occurrence matrix\n"
xlabel= ''#"Labels"
ylabel= ''#"Labels"
xticklabels = label_header_with_count
yticklabels = label_header_with_count
heatmap(cooccurrence_matrix_percentage, title, xlabel, ylabel, xticklabels, yticklabels)
plt.savefig('image_output.png', dpi=300, format='png', bbox_inches='tight') # use format='svg' or 'pdf' for vectorial pictures
#plt.show()

In [ ]:
def voc_size():
  new= df['text'].str.split()
  new=new.values.tolist()
  corpus=[word for i in new for word in i]
  corpus = set(corpus)
  print(len(corpus))

voc_size()

In [ ]:
import numpy as np


In [ ]:
np.median(np.array(df['text'].str.len()))

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('Number of chars')
plt.ylabel('frequency')
df['text'].str.len().hist()

In [ ]:
plt.boxplot(list(df['text'].str.len()))


plt.savefig('SimpleBoxPlot.png')
plt.show()

In [ ]:
import numpy as np
arr1  =list(df['text'].str.len())
q1 = np.quantile(arr1, 0.25)
 
# finding the 3rd quartile
q3 = np.quantile(arr1, 0.75)
med = np.median(arr1)
 
# finding the iqr region
iqr = q3-q1
 
# finding upper and lower whiskers
upper_bound = q3+(1.5*iqr)
lower_bound = q1-(1.5*iqr)
print(iqr, upper_bound, lower_bound)

In [ ]:
plt.xlabel('Number of words')
plt.ylabel('frequency')
df['text'].apply(lambda x : len(x.split(' '))).hist()

In [ ]:
import numpy as np
arr1  =list(df['text'].apply(lambda x : len(x.split(' '))))
q1 = np.quantile(arr1, 0.25)
 
# finding the 3rd quartile
q3 = np.quantile(arr1, 0.75)
med = np.median(arr1)
 
# finding the iqr region
iqr = q3-q1
 
# finding upper and lower whiskers
upper_bound = q3+(1.5*iqr)
lower_bound = q1-(1.5*iqr)
print(iqr, upper_bound, lower_bound)

In [ ]:
df[(df['text'].apply(lambda x : len(x.split(' '))) >= upper_bound)].index

In [ ]:
len(set([  1,  14,  17,  36,  40,  53,  55,  70,  72,  78,  81, 111, 118,
            129, 153, 193, 219, 226, 243, 264, 270, 280, 306, 307, 309, 312,
            336, 420, 422, 425, 449]).intersection(set([ 14,  17,  36,  53,  55,  70,  72,  78, 111, 118, 129, 142, 153,
            172, 193, 219, 226, 264, 270, 280, 306, 307, 309, 312, 316, 336,
            420, 422, 425, 449]) ))

In [ ]:
df[(arr1 >= upper_bound)].index

In [ ]:
df['text'].apply(lambda x : len(x) >= 2000 and len(x) <= 10000).sum() / len(df)

In [ ]:
np.median(np.array(df['text'].str.split().\
    map(lambda x: len(x))))

In [ ]:
df['text'].str.split().\
    map(lambda x: len(x)).\
    hist()

In [ ]:
df['text'].str.split().\
    map(lambda x: len(x) <= 100).sum() /len(df)

In [ ]:
### correlation

import numpy as np

np.corrcoef(df['text'].str.split(). map(lambda x: len(x)), df['text'].str.len())[0,1]

In [ ]:
from scipy.stats.stats import pearsonr

pearsonr(df['text'].str.split(). map(lambda x: len(x)), df['text'].str.len())

In [ ]:
plt.xlabel('Average word length')
plt.ylabel('frequency')
df['text1'].str.split().\
   apply(lambda x : [len(i) for i in x]). \
   map(lambda x: np.mean(x)).hist()

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize 
import nltk
nltk.download('punkt')
df['text'].apply(lambda x : len(sent_tokenize(x))).hist()
plt.xlabel('Number of sentences')
plt.ylabel('Frequency')

In [ ]:
(df['text'].apply(lambda x : len(sent_tokenize(x))) <= 10).sum() / len(df)

In [ ]:
import numpy as np
arr1  =list(df['text'].apply(lambda x : len(sent_tokenize(x))))
q1 = np.quantile(arr1, 0.25)
 
# finding the 3rd quartile
q3 = np.quantile(arr1, 0.75)
med = np.median(arr1)
 
# finding the iqr region
iqr = q3-q1
 
# finding upper and lower whiskers
upper_bound = q3+(1.5*iqr)
lower_bound = q1-(1.5*iqr)
print(iqr, upper_bound, lower_bound)

In [ ]:
(arr1 >= upper_bound).sum() / len(df)

In [ ]:
# deal with contractions
# use this tokenization wordpunct_tokenize

In [ ]:
def gogo(text):
  value = []
  for x in sent_tokenize(text):
    value.append(len(x.split(' ')))
    print(x.split(' '))
  return np.mean(value)

df['text'].apply(gogo).hist()

In [ ]:
from nltk.tokenize import wordpunct_tokenize
for x in df['text'].apply(lambda x : wordpunct_tokenize(x)).apply(lambda x : max(x, key=len)):
  print(x, len(x))

In [ ]:
df['text'][3]

In [ ]:
df['text'].str.split()

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


In [ ]:
import spacy    
nlp = spacy.load("en")

In [ ]:
import matplotlib.pyplot as plt 
def plot_top_stopwords_barchart(text, stop):
    new= text.str.split()
    new=new.values.tolist()
    corpus=[word for i in new for word in i]
    from collections import defaultdict
    dic=defaultdict(int)
    for word in corpus:
        if word in stop:
            dic[word]+=1
            
    top=sorted(dic.items(), key=lambda x:x[1],reverse=True)[:10] 
    x,y=zip(*top)
    plt.bar(x,y)
    plt.xlabel('stop words')
    plt.ylabel('frequency')

spacy_stop = set(nlp.Defaults.stop_words)
stop=set(stopwords.words('english'))
plot_top_stopwords_barchart(df['text'], spacy_stop)

In [ ]:

# Code Snippet for Top Non-Stopwords Barchart

import seaborn as sns
from nltk.corpus import stopwords
from collections import  Counter
nltk.download('stopwords')
def plot_top_non_stopwords_barchart(text):
    stop=set(stopwords.words('english'))
    
    new= text.str.split()
    new=new.values.tolist()
    corpus=[word for i in new for word in i]

    counter=Counter(corpus)
    most=counter.most_common()
    x, y=[], []
    for word,count in most:
        if (word not in stop):
            x.append(word)
            y.append(count)

    x = x[:20] 
    y = np.array(y[:20]) / len(counter)     
    sns.barplot(x=y,y=x)

plot_top_non_stopwords_barchart(df['text'])


In [ ]:
df['text']

In [ ]:
from nltk.util import ngrams
from  sklearn.feature_extraction.text  import CountVectorizer

def get_top_ngram(corpus, n=None):
    
    vec = CountVectorizer(ngram_range=(n, n)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) 
                  for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:20]

top_n_bigrams=get_top_ngram(df['text'], 2)[:20] 
x,y=map(list,zip(*top_n_bigrams))
sns.barplot(x=y,y=x)

In [ ]:
import gensim
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

def preprocess_news(df):
    corpus=[]
    
    lem=WordNetLemmatizer()
    for news in df['text']:
        words=[w for w in word_tokenize(news) if (w not in stop)]
        
        words=[lem.lemmatize(w) for w in words if len(w)>2]
        
        corpus.append(words)
    return corpus

corpus=preprocess_news(df)

dic=gensim.corpora.Dictionary(corpus)
bow_corpus = [dic.doc2bow(doc) for doc in corpus]

In [ ]:
df['text'].apply(lambda x : sent_tokenize(x))

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 14, 
                                   id2word = dic,                                    
                                   passes = 500,
                                   workers = 2)
lda_model.show_topics()




In [ ]:
!pip install pyLDAvis

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, bow_corpus, dic)
vis

In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

In [ ]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=corpus, dictionary=dic, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\\nCoherence Score: ', coherence_lda)

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = num_topics, 
                                   id2word = dic,                                    
                                   passes = 200,
                                   workers = 3)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

limit=20; start=2; step=1;

model_list, coherence_values = compute_coherence_values(dictionary=dic, corpus=corpus, texts=corpus, start=start, limit=limit, step=step)

# Show graph
x = range(start, limit, step)
plt.plot(x, coherence_values, start, limit, step)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
g = np.argmax(coherence_values)

In [ ]:
g

In [ ]:
coherence_values[g]

In [ ]:
plt.scatter(x, coherence_values, c ="blue")
plt.scatter(g, coherence_values[g], c = 'red')
plt.xticks(np.arange(0, 20, 2))
plt.xlabel('number topics')
plt.ylabel('coherence value')
# To show the plot
plt.show()

In [ ]:
coherence_values

In [ ]:
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)

def show_wordcloud(data):
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stopwords,
        max_words=150,
        max_font_size=30,
        scale=3,
        random_state=5)
   
    wordcloud=wordcloud.generate(str(data))

    fig = plt.figure(1, figsize=(12, 12))
    plt.axis('off')

    plt.imshow(wordcloud)
    plt.show()

show_wordcloud(df['text'])

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [ ]:
def ner(text):
    doc=nlp(text)
    return [X.label_ for X in doc.ents]

ent=df['text'].\
    apply(lambda x : ner(x))
ent=[x for sub in ent for x in sub]

counter=Counter(ent)
count=counter.most_common()

In [ ]:
x,y=map(list,zip(*count))
sns.barplot(x=y,y=x)

In [ ]:
sum([x[1] for x in count])

In [ ]:
(235 + 151) / 792

In [ ]:
count

In [ ]:
nlp("temazapan is good").ents[0].label_

In [ ]:
def ner(text,ent="GPE"):
    doc=nlp(text)
    return [X.text for X in doc.ents if X.label_ == ent]

gpe=df['text'].apply(lambda x: ner(x))
gpe=[i for x in gpe for i in x]
counter=Counter(gpe)

x,y=map(list,zip(*counter.most_common(10)))
sns.barplot(y,x)

In [ ]:
# Code Snippet for Parts of Speach Barchart

import nltk
from nltk.tokenize import word_tokenize
import seaborn as sns
from collections import Counter

def plot_parts_of_speach_barchart(text):
    nltk.download('averaged_perceptron_tagger')

    def _get_pos(text):
        pos=nltk.pos_tag(word_tokenize(text))
        pos=list(map(list,zip(*pos)))[1]
        return pos
    
    tags=text.apply(lambda x : _get_pos(x))
    tags=[x for l in tags for x in l]
    counter=Counter(tags)
    x,y=list(map(list,zip(*counter.most_common(5))))
    
    sns.barplot(x=y,y=x)

plot_parts_of_speach_barchart(df['text'])


In [ ]:
def get_adjs(text):
    adj=[]
    pos=nltk.pos_tag(word_tokenize(text))
    for word,tag in pos:
        if tag=='JJ':
            adj.append(word)
    return adj


words=df['text'].apply(lambda x : get_adjs(x))
words=[x for l in words for x in l]
counter=Counter(words)

x,y=list(map(list,zip(*counter.most_common(5))))
sns.barplot(x=y,y=x)

In [ ]:
from textstat import flesch_reading_ease

df['text'].\
   apply(lambda x : flesch_reading_ease(x)).hist()

In [ ]:
reading = df['text'].\
   apply(lambda x : flesch_reading_ease(x))
x=[i for i in range(len(reading)) if reading[i]<10]

df.iloc[x]['text'].head()

In [ ]:
df.iloc[185]['text']

In [ ]:
df.iloc[116]['text']

In [ ]:
from nltk.tokenize import sent_tokenize,wordpunct_tokenize
import numpy as np
import nltk
nltk.download('punkt')

c=[]
def okok(text):
  c = []
  for x in sent_tokenize(text):
    c.append(len((wordpunct_tokenize(x))))
  return np.mean(c)

df['text'].apply(okok).hist()
plt.xlabel('Average sentence length/ Document')
plt.ylabel('Frequency')

In [ ]:
import numpy as np
arr1  =list(df['text'].apply(okok))
q1 = np.quantile(arr1, 0.25)
 
# finding the 3rd quartile
q3 = np.quantile(arr1, 0.75)
med = np.median(arr1)
 
# finding the iqr region
iqr = q3-q1
 
# finding upper and lower whiskers
upper_bound = q3+(1.5*iqr)
lower_bound = q1-(1.5*iqr)
print(iqr, upper_bound, lower_bound)

In [ ]:
(df['text'].apply(okok) > upper_bound).sum()

In [ ]:
np.median(c)

In [ ]:
(df['text'].apply(okok) < 30).sum() / len(df)

In [ ]:
(df['text'].apply(sent_tokenize).apply(len) > 30).sum() 

In [ ]:
ans = []
for s in df['text'].apply(sent_tokenize):
  number = []
  for x in s:
    number.append(len(word_tokenize(x)))
  ans.append(sum(number) / len(number))


plt.hist(ans)

In [ ]:
(np.array(ans) < 20).astype(int).sum() / len(df)

In [ ]:
ans = []
for s in df['text'].apply(sent_tokenize):

  for x in s:
    ans.append(len(word_tokenize(x)))


plt.hist(ans)

In [ ]:
ans = np.array(ans)

In [ ]:
ans.mean()

In [ ]:
ans = []
for s in df['text'].apply(sent_tokenize):
  ans.append(len(s))

plt.hist(ans)

In [ ]:
plt.hist(ans)

In [ ]:
ans = np.array(ans)

In [ ]:
ans.max()

In [ ]:
np.median(np.array(df['text'].apply(sent_tokenize).apply(len)))

In [ ]:
from scipy.stats.stats import pearsonr

pearsonr(df['text'].apply(sent_tokenize).apply(len), df['text'].str.len())